In [1]:
import pandas as pd
import os
import numpy as np

from sklearn.datasets import fetch_20newsgroups
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [7]:
path=os.getcwd()+'\\BERT-NER'
notebook_path=os.getcwd()
data_path=os.getcwd()[:-9]+'data'

In [8]:
!git clone https://github.com/kamalkraj/BERT-NER.git

fatal: could not create work tree dir 'os.getcwd()[:-9]': Invalid argument


In [14]:
os.chdir(path)

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
!python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --task_name=ner --output_dir=out_ner --max_seq_length=128 --do_train --num_train_epochs 3 --do_eval --warmup_proportion=0.1

In [ ]:
#Download the BERT model and finetune it on Conll2003.

%%writefile bert.py
"""BERT NER Inference."""

from __future__ import absolute_import, division, print_function

import json
import os

import torch
import torch.nn.functional as F
from nltk import word_tokenize
from pytorch_transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)


class BertNer(BertForTokenClassification):

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, valid_ids=None):
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask, head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32,device='cuda' if torch.cuda.is_available() else 'cpu')
        for i in range(batch_size):
            jj = -1
            for j in range(max_len):
                    if valid_ids[i][j].item() == 1:
                        jj += 1
                        valid_output[i][jj] = sequence_output[i][j]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)
        return logits

class Ner:

    def __init__(self,model_dir: str):
        self.model , self.tokenizer, self.model_config = self.load_model(model_dir)
        self.label_map = self.model_config["label_map"]
        self.max_seq_length = self.model_config["max_seq_length"]
        self.label_map = {int(k):v for k,v in self.label_map.items()}
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device)
        self.model.eval()

    def load_model(self, model_dir: str, model_config: str = "model_config.json"):
        model_config = os.path.join(model_dir,model_config)
        model_config = json.load(open(model_config))
        model = BertNer.from_pretrained(model_dir)
        tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=model_config["do_lower"])
        return model, tokenizer, model_config

    def tokenize(self, text: str):
        """ tokenize input"""
        words = word_tokenize(text)
        tokens = []
        valid_positions = []
        for i,word in enumerate(words):
            token = self.tokenizer.tokenize(word)
            tokens.extend(token)
            for i in range(len(token)):
                if i == 0:
                    valid_positions.append(1)
                else:
                    valid_positions.append(0)
        return tokens, valid_positions

    def preprocess(self, text: str):
        """ preprocess """
        tokens, valid_positions = self.tokenize(text)
        ## insert "[CLS]"
        tokens.insert(0,"[CLS]")
        valid_positions.insert(0,1)
        ## insert "[SEP]"
        tokens.append("[SEP]")
        valid_positions.append(1)
        segment_ids = []
        for i in range(len(tokens)):
            segment_ids.append(0)
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        while len(input_ids) < self.max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            valid_positions.append(0)
        return input_ids,input_mask,segment_ids,valid_positions

    def predict(self, text: str):
        input_ids,input_mask,segment_ids,valid_ids = self.preprocess(text)
        input_ids = torch.tensor([input_ids],dtype=torch.long,device=self.device)
        input_mask = torch.tensor([input_mask],dtype=torch.long,device=self.device)
        segment_ids = torch.tensor([segment_ids],dtype=torch.long,device=self.device)
        valid_ids = torch.tensor([valid_ids],dtype=torch.long,device=self.device)
        with torch.no_grad():
            logits = self.model(input_ids, segment_ids, input_mask,valid_ids)
        logits = F.softmax(logits,dim=2)
        logits_label = torch.argmax(logits,dim=2)
        logits_label = logits_label.detach().cpu().numpy().tolist()[0]

        logits_confidence = [values[label].item() for values,label in zip(logits[0],logits_label)]

        logits = []
        pos = 0
        for index,mask in enumerate(valid_ids[0]):
            if index == 0:
                continue
            if mask == 1:
                logits.append((logits_label[index-pos],logits_confidence[index-pos]))
            else:
                pos += 1
        logits.pop()

        labels = [(self.label_map[label],confidence) for label,confidence in logits]
        words = word_tokenize(text)
        assert len(labels) == len(words)

        Person = []
        Location = []
        Organization = []
        Miscelleneous = []

        for word, (label, confidence) in zip(words, labels):
            if label=="B-PER" or label=="I-PER":
                Person.append(word)
            elif label=="B-LOC" or label=="I-LOC":
                Location.append(word)
            elif label=="B-ORG" or label=="I-ORG":
                Organization.append(word)
            elif label=="B-MISC" or label=="I-MISC":
                Miscelleneous.append(word)
            else:
                output = None

        output = []
        for word, (label, confidence) in zip(words, labels):      
            if label == "B-PER":
                output.append(' '.join(Person) + ": Person")
            if label=="B-LOC":
                output.append(' '.join(Location) + ": Location")
            if label=="B-MISC":
                output.append(' '.join(Miscelleneous) + ": Miscelleneous Entity")
            if label=="B-ORG":
                output.append(' '.join(Organization) + ": Organization")
                
        return output

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from bert import Ner
model = Ner("out_ner/")

In [11]:
os.chdir(data_path)
tf=pd.read_csv('summary_sentences.csv')


In [ ]:
def predict_NE(values,model):
    '''
    Predict the NE results from values using model
    '''
    ne=[]
    for sentence in values:
        ne.append(model.predict(sentence))
    return ne

### 5 NEWS GROUP

In [ ]:
ne=predict_NE(tf.summary_sentences.values,model)
tf['NE']=ne

for i in ne: 
    for j in range(len(i)):
        i[j]=i[j].split(':')[0]

#Save to local csv
tf.to_csv('Summary_Sentences_NE.csv',index=False)

### 20 NEWS GROUP

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='train')
news_df = pd.DataFrame.from_dict(newsgroups_train,'index').T
news_df.data = news_df.data.apply(lambda x: clean_text(x)+ '.')
news_df=news_df.reset_index()
news_df['summary_sentences']=news_df.data.apply(lambda x:split_into_sentences(x))
df_summary_sentences=news_df.summary_sentences.explode().reset_index()
df_summary_sentences=df_summary_sentences.merge(news_df[['index','target']], on="index", how = 'left')

In [7]:
def predict_largeNE(values,model):
    ne2=[]
    for sentence in values:
        try:
            if len(sentence)<512:
                ne2.append(model.predict(sentence))
            else:
                ne2.append([])
        except (AssertionError,TypeError):
            ne2.append([])
    return ne2

In [11]:
ne2=predict_largeNE(df_summary_sentences.summary_sentences.values,model)

In [12]:
df_summary_sentences['NE']=ne2
df_summary_sentences.to_csv('raw.csv',index=False)